# Задача:
Компания купила крупный сервис для чтения книг по подписке. <br>
Моя первая задача как аналитика — проанализировать базу данных. <br>
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

# Описание данных:

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

# Подготовка

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [1]:
# устанавливаем параметры
# db_config = {'user': 'Какое-то имя', # имя пользователя
# 'pwd': 'Какой-то пароль', # пароль
# 'host': 'Какой-то хост',
# 'port': Какой-то порт, # порт подключения
# 'db': 'ещё что-то'} # название базы данных

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

In [4]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [5]:
def sql(query):
    return(pd.io.sql.read_sql(query, con = engine))

# Исследуйте таблицы — выведите первые строки;

## books

In [6]:
sql('''
    SELECT *
    FROM books
    LIMIT 1
''')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


## authors

In [7]:
sql('''
    SELECT *
    FROM authors
    LIMIT 1
''')

,author_id,author
0,1,A.S. Byatt


## publishers

In [8]:
sql('''
    SELECT *
    FROM publishers
    LIMIT 1
''')

,publisher_id,publisher
0,1,Ace


## ratings

In [9]:
sql('''
    SELECT *
    FROM ratings
    LIMIT 1
''')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


## reviews

In [10]:
sql('''
    SELECT *
    FROM reviews
    LIMIT 1
''')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


# Задания

## Посчитайте, сколько книг вышло после 1 января 2000 года;

In [11]:
sql('''
    SELECT COUNT(*)
    FROM books
    WHERE publication_date > '2000-01-01'
''')

,count
0,819


Вывод:<br>

Всего 819 книг вышло после 1 января 2000 года.

## Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [12]:
sql('''

    SELECT book_id,
            title,
            ROUND(AVG(rating),2) AS avg_rating,
            COUNT(DISTINCT (review_id)) AS count_reviews
    FROM books
        LEFT JOIN ratings USING(book_id)
        LEFT JOIN reviews USING(book_id)
    GROUP BY book_id
    ORDER BY 4
''')

,book_id,title,avg_rating,count_reviews
0,83,Anne Rice's The Vampire Lestat: A Graphic Novel,3.67,0
1,672,The Cat in the Hat and Other Dr. Seuss Favorites,5.00,0
2,221,Essential Tales and Poems,4.00,0
3,808,The Natural Way to Draw,3.00,0
4,191,Disney's Beauty and the Beast (A Little Golden...,4.00,0
...,...,...,...,...
995,733,The Giver (The Giver #1),3.75,6
996,854,The Road,3.77,6
997,695,The Curious Incident of the Dog in the Night-Time,4.08,6
998,696,The Da Vinci Code (Robert Langdon #2),3.83,6


Вывод:<br>

Встречаются книги без обзоров.<br>
Достал данные, добавил новые столбцы: 
<ul>
   <li> avg_rating - средний рейтинг книги
       <li>count_reviews - количество обзоров.
</ul>

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

Условие в HAVING - при варианте, если окажется несколько издательств с одинаковым количеством выпущеных книг.

In [13]:
sql('''
    WITH id AS (
        SELECT publisher_id,COUNT(book_id) AS count_books
        FROM books
        WHERE num_pages > 50
        GROUP BY publisher_id
        HAVING COUNT(book_id) = (
                            SELECT COUNT(book_id)
                            FROM books
                            WHERE num_pages > 50
                            GROUP BY publisher_id
                            ORDER BY 1 DESC
                            LIMIT 1
                            )
)
    SELECT publisher,count_books
    FROM id
        INNER JOIN publishers USING (publisher_id)
''')

,publisher,count_books
0,Penguin Books,42


Вывод:<br>

Издательство *Penguin Books* выпустило наибольшее число книг толще 50 страниц.

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [14]:
sql('''
    SELECT author,ROUND(AVG(rating),2) AS avg_rating
    FROM ratings
        INNER JOIN books USING(book_id)
        INNER JOIN authors USING(author_id)
    WHERE book_id IN (
                    SELECT book_id
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(rating_id) >= 50 ) 
    GROUP BY author
    ORDER BY AVG(rating) DESC
    LIMIT 1
''')

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


Вывод: <br>

Самый популярный автор - дуэт J.K. Rowling и Mary GrandPré со средним рейтингом 4.29.

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [15]:
sql('''
SELECT (COUNT(DISTINCT review_id)::NUMERIC /  COUNT(DISTINCT username)) AS avg_reviews
FROM (
    SELECT username,COUNT(rating_id)
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) > 50
    ) AS subq 
        LEFT JOIN reviews USING(username)
''')

,avg_reviews
0,24.333333


Вывод: 

Пользователи, которые поставили больше 50 оценок, в среднем пишут по 24 обзора.

# Вывод:
<ol>
    <li>Посчитайте, сколько книг вышло после 1 января 2000 года; </li>
        
        Всего 819 книг вышло после 1 января 2000 года

    
    
   <li> Для каждой книги посчитайте количество обзоров и среднюю оценку.
    
        Были добавлены новые столбцы с количеством обзоров и средней оценкой.
   </li>
    <li>Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;¶
        
        Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц.
   <li>Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
        
        Самый популярный автор - дуэт J.K. Rowling и Mary GrandPré со средним рейтингом 4.29.
   <li>Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
       
       Пользователи, которые поставили больше 50 оценок, в среднем пишут по 24 обзора.
        
</ol>    

В ходе работы я смог подключиться к БД, извлечь данные, составить простые запросы, ответить на поставленные вопросы.